In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import img_to_array, array_to_img
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
image_path='/content/drive/MyDrive/Images Dataset/unsplash-images-collection'

In [ ]:
# Set random seed for reproducibility
np.random.seed(42)

In [ ]:
#Data Preprocessing
def load_and_preprocess_images(image_path, img_size=(128, 128)):
    """
    Load and preprocess images from the dataset.
    Args:
        dataset_path (str): Path to the dataset directory.
        img_size (tuple): Target image size (height, width).
    Returns:
        np.array: Preprocessed images as a numpy array.
    """
    images = []
    for img_name in os.listdir(image_path):
        img_path = os.path.join(image_path, img_name)
        try:
            # Load image
            img = Image.open(img_path).convert('L')  # Convert to grayscale
            img = img.resize(img_size)  # Resize image
            img = img_to_array(img)  # Convert to numpy array
            img = img / 255.0  # Normalize to [0, 1]
            images.append(img)
        except Exception as e:
            print(f"Skipping {img_name}: {e}")
    return np.array(images)

In [ ]:
#Traditional Image Enhancement Techniques
def apply_traditional_enhancement(image):
    """
    Apply traditional image enhancement techniques.
    Args:
        image (np.array): Input image.
    Returns:
        np.array: Enhanced image.
    """
    # Histogram Equalization
    image = cv2.equalizeHist((image * 255).astype(np.uint8))
    # Gaussian Blurring
    image = cv2.GaussianBlur(image, (5, 5), 0)
    return image / 255.0

In [ ]:
# Deep Learning-Based Image Enhancement (Autoencoder)
def build_autoencoder(input_shape=(128, 128, 1)):
    """
    Build a convolutional autoencoder for image enhancement.
    Args:
        input_shape (tuple): Input image shape (height, width, channels).
    Returns:
        Model: Autoencoder model.
    """
    input_img = Input(shape=input_shape) # Define the input tensor
    # Encoder
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img) # Pass the input tensor here
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    encoded = MaxPooling2D((2, 2), padding='same')(x)
     # Decoder
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(encoded)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

    # Autoencoder model
    autoencoder = Model(input_img, decoded)
    autoencoder.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
    return autoencoder

In [ ]:
GAN-Based Super-Resolution (Simplified)
def build_gan(input_shape=(128, 128, 1)):
    """
    Build a simplified GAN for super-resolution.
    Args:
        input_shape (tuple): Input image shape (height, width, channels).
    Returns:
        tuple: Generator and Discriminator models.
    """
    # Generator
    input_img = Input(shape=input_shape)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(input_img)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)
    generator = Model(input_img, x)

    # Discriminator
    input_img_disc = Input(shape=input_shape)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(input_img_disc)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)
    discriminator = Model(input_img_disc, x)

    # Compile Discriminator
    discriminator.compile(optimizer=Adam(learning_rate=0.0002), loss='binary_crossentropy')

    return generator, discriminator

SyntaxError: invalid syntax (<ipython-input-33-cb743f7e28d8>, line 1)

In [ ]:
#GAN-Based Super-Resolution (Simplified)
def build_gan(input_shape=(128, 128, 1)):
    """
    Build a simplified GAN for super-resolution.
    Args:
        input_shape (tuple): Input image shape (height, width, channels).
    Returns:
        tuple: Generator and Discriminator models.
    """
    # Generator
    input_img = Input(shape=input_shape)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(input_img)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)
    generator = Model(input_img, x)

    # Discriminator
    input_img_disc = Input(shape=input_shape)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(input_img_disc)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)
    discriminator = Model(input_img_disc, x)

    # Compile Discriminator
    discriminator.compile(optimizer=Adam(learning_rate=0.0002), loss='binary_crossentropy')

    return generator, discriminator



In [ ]:
#Evaluation Metrics
def evaluate_image_quality(original, enhanced):
    """
    Evaluate image quality using PSNR and SSIM.
    Args:
        original (np.array): Original image.
        enhanced (np.array): Enhanced image.
    Returns:
        tuple: PSNR and SSIM scores.
    """
    psnr_score = psnr(original, enhanced, data_range=1)
    ssim_score = ssim(original, enhanced, data_range=1, win_size=3)
    return psnr_score, ssim_score#

In [ ]:
if __name__ == "__main__":
    # Load dataset
    # image_path = "unsplash-images-collection"  # This was the incorrect relative path
    image_path = '/content/drive/MyDrive/Images Dataset/unsplash-images-collection' # Use the full path instead
    images = load_and_preprocess_images(image_path=image_path) # Pass the full path to the function

    # Build and apply the autoencoder
    autoencoder = build_autoencoder()  # Create an instance of the autoencoder
    enhanced_autoencoder = autoencoder.predict(images) # Enhance the images

# Apply traditional enhancement
enhanced_traditional = np.array([apply_traditional_enhancement(img) for img in images])

# Train Autoencoder
autoencoder = build_autoencoder()
autoencoder.fit(images, images, epochs=50, batch_size=32, validation_split=0.2, callbacks=[EarlyStopping(patience=3)])
enhanced_autoencoder = autoencoder.predict(images)

# Train GAN (Simplified)
generator, discriminator = build_gan()
# Note: GAN training is simplified for demonstration purposes.

# Evaluate and visualize results
for i in range(min(5, len(images))):  # Display first 5 images
    original = images[i]
    traditional = enhanced_traditional[i]
    autoencoded = enhanced_autoencoder[i]

    # Evaluate quality
    psnr_traditional, ssim_traditional = evaluate_image_quality(original, traditional)
    psnr_autoencoded, ssim_autoencoded = evaluate_image_quality(original, autoencoded)

26/26 ━━━━━━━━━━━━━━━━━━━━ 15s 559ms/step
Epoch 1/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 50s 2s/step - loss: 0.0722 - val_loss: 0.0208
Epoch 2/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 81s 2s/step - loss: 0.0168 - val_loss: 0.0107
Epoch 3/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 82s 2s/step - loss: 0.0097 - val_loss: 0.0072
Epoch 4/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 46s 2s/step - loss: 0.0075 - val_loss: 0.0063
Epoch 5/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 46s 2s/step - loss: 0.0065 - val_loss: 0.0056
Epoch 6/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 46s 2s/step - loss: 0.0059 - val_loss: 0.0056
Epoch 7/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 82s 2s/step - loss: 0.0055 - val_loss: 0.0048
Epoch 8/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 82s 2s/step - loss: 0.0049 - val_loss: 0.0042
Epoch 9/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 49s 2s/step - loss: 0.0044 - val_loss: 0.0040
Epoch 10/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 46s 2s/step - loss: 0.0042 - val_loss: 0.0038
Epoch 11/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 82s 2s/step - loss: 0.0041 - val_loss: 0.0040
Epoch 12/50
21/21 ━━━━━━━━━━━━━━

ValueError: win_size exceeds image extent. Either ensure that your images are at least 7x7; or pass win_size explicitly in the function call, with an odd value less than or equal to the smaller side of your images. If your images are multichannel (with color channels), set channel_axis to the axis number corresponding to the channels.

In [ ]:
def apply_traditional_enhancement(image):
    """
    Apply traditional image enhancement techniques.
    Args:
        image (np.array): Input image.
    Returns:
        np.array: Enhanced image.
    """
    # Histogram Equalization
    image = cv2.equalizeHist((image * 255).astype(np.uint8))
    # Gaussian Blurring
    image = cv2.GaussianBlur(image, (5, 5), 0)

    # Reshape to add channel dimension
    image = image[..., np.newaxis] / 255.0  # Add channel dimension and normalize

    return image # Ensure the enhanced image also has a channel dimension

In [ ]:
# Apply traditional enhancement
enhanced_traditional = np.array([apply_traditional_enhancement(img) for img in images])

# Train Autoencoder
autoencoder = build_autoencoder()
autoencoder.fit(images, images, epochs=50, batch_size=32, validation_split=0.2, callbacks=[EarlyStopping(patience=3)])
enhanced_autoencoder = autoencoder.predict(images)

# Train GAN (Simplified)
generator, discriminator = build_gan()
# Note: GAN training is simplified for demonstration purposes.

# Evaluate and visualize results
for i in range(min(5, len(images))):  # Display first 5 images
    original = images[i]
    traditional = enhanced_traditional[i]
    autoencoded = enhanced_autoencoder[i]

    # Evaluate quality
    psnr_traditional, ssim_traditional = evaluate_image_quality(original, traditional)
    psnr_autoencoded, ssim_autoencoded = evaluate_image_quality(original, autoencoded)

Epoch 1/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 49s 2s/step - loss: 0.0703 - val_loss: 0.0165
Epoch 2/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 47s 2s/step - loss: 0.0132 - val_loss: 0.0077
Epoch 3/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 50s 2s/step - loss: 0.0079 - val_loss: 0.0073
Epoch 4/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 80s 2s/step - loss: 0.0073 - val_loss: 0.0064
Epoch 5/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 82s 2s/step - loss: 0.0065 - val_loss: 0.0062
Epoch 6/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 49s 2s/step - loss: 0.0059 - val_loss: 0.0053
Epoch 7/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 82s 2s/step - loss: 0.0056 - val_loss: 0.0047
Epoch 8/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 79s 2s/step - loss: 0.0048 - val_loss: 0.0046
Epoch 9/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 82s 2s/step - loss: 0.0047 - val_loss: 0.0045
Epoch 10/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 82s 2s/step - loss: 0.0044 - val_loss: 0.0041
Epoch 11/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 82s 2s/step - loss: 0.0042 - val_loss: 0.0039
Epoch 12/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 82s 2s/step - loss: 0.0041 - val_lo

ValueError: win_size exceeds image extent. Either ensure that your images are at least 7x7; or pass win_size explicitly in the function call, with an odd value less than or equal to the smaller side of your images. If your images are multichannel (with color channels), set channel_axis to the axis number corresponding to the channels.

In [ ]:
def evaluate_image_quality(original, enhanced):
    """
    Evaluate image quality using PSNR and SSIM.
    Args:
        original (np.array): Original image.
        enhanced (np.array): Enhanced image.
    Returns:
        tuple: PSNR and SSIM scores.
    """
    psnr_score = psnr(original, enhanced, data_range=1)

    # Adjust win_size based on image dimensions
    win_size = min(7, min(original.shape[0], original.shape[1]))  # Set to 7 or smaller
    if win_size % 2 == 0:  # Ensure win_size is odd
        win_size -= 1

    ssim_score = ssim(original, enhanced, data_range=1, win_size=win_size)
    return psnr_score, ssim_score

In [ ]:
def visualize_results(original, traditional, autoencoded):
    """Visualizes the results of image enhancement techniques.
    Args:
        original (np.array): Original image.
        traditional (np.array): Traditionally enhanced image.
        autoencoded (np.array): Autoencoder enhanced image.
    """
    plt.figure(figsize=(15, 5))
    plt.subplot(1, 3, 1)
    plt.imshow(original.squeeze(), cmap='gray')
    plt.title("Original Image")

    plt.subplot(1, 3, 2)
    plt.imshow(traditional.squeeze(), cmap='gray')
    plt.title(f"Traditional\nPSNR: {psnr_traditional:.2f}, SSIM: {ssim_traditional:.2f}")

    plt.subplot(1, 3, 3)
    plt.imshow(autoencoded.squeeze(), cmap='gray')
    plt.title(f"Autoencoder\nPSNR: {psnr_autoencoded:.2f}, SSIM: {ssim_autoencoded:.2f}")

    plt.show()